This notebook involves the implementation of a simple vector search system.

It involves:

1. loading and processing a csv dataset
2. Vectorizing the text data using a huggingface embedding model
3. Storing the vectorized data on an opensource vector database called Chromadb.
4. Performing similarity searches on the vectorized data.

In [ ]:
# Installing required libraries: chroma, langchain, etc.
!pip install chromadb
!pip install langchain sentence-transformers langchain_chroma
!pip install langchain_community
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 87.4 MB/s eta 0:00:0

In [ ]:
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
from langchain.docstore.document import Document  # Import the Document class

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### 1. Loading and preprocessing the dataset:

*    We only include the 1000 lines of the dataset for now, for simplicity and speed
*   in the next block, we turn the dataframe to list, and then Document object in langchain (which stores each row's content as page_content)



In [ ]:
df_total = pd.read_csv("cleaned_TEP-fa.csv")
df_total = df_total [["English"]]
df = df_total.head(10000)

In [ ]:
# Assuming the sentences are stored in a column called 'English'
documents = df['English'].tolist()

# Create a list of Document objects
document_objects = [Document(page_content=doc) for doc in documents]

### 2.& 3. Vectorizing the text data using a huggingface embedding model and Storing on Chromadb.

Now we need an embedding model! Here we chose the sentence-transformer "all-mpnet-base-v2" model from huggingface, which maps sentences and paragraphs to a 768 dimensional dense vector space.
- More about this this model: [here](https://https://huggingface.co/sentence-transformers/all-mpnet-base-v2)
- SentenceTransformers documentation: [here](https://www.sbert.net/)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
embed_model = HuggingFaceEmbeddings(model_name = model_name)

# Create a vector database and add the vectorized documents
vector_db = Chroma.from_documents(document_objects, embed_model)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 4. Performing similarity searches on the vectorized data


*   We define a function `search_similar` that takes in parameters k and query, and returnes the top k documents most similar to the query.



In [ ]:
# Function to search for similar sentences based on a query
def search_similar(query, k=10):
    similar_docs = vector_db.similarity_search(query, k=k)
    return similar_docs

In [ ]:
# Example query
query = "movies are an excellent story telling medium"
results = search_similar(query)

In [ ]:
# Print the results
for result in results:
    print(result.page_content)  # Access the content of each Document

people want human interest stories
people want human interest stories
can i read you a story
can i read you a story
you wrote me a play
you wrote me a play
bringing with it the experiences
bringing with it the experiences
about the supernatural i know
about the supernatural i know


**Additional information:** the TEP dataset which was used in this notebook, has multiple double entries, hence the double documents in results!